In [1]:
import os
import configparser
import requests
import boto3
import json
import re
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from timeit import default_timer as timer
from datetime import timedelta

os.environ['AWS_ACCESS_KEY_ID']="AKKHU4FVNX"
os.environ['AWS_SECRET_ACCESS_KEY']="Ur9oXp0fY9/80s3"

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1609527416191_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def create_spark_session():
    """
    Creates the spark session used in all the code

    """
    spark = SparkSession \
        .builder \
        .appName("Udacity Final Project") \
        .getOrCreate()
    return spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def load_csv(sparksession, path=[], header=True, headerList=[]):    
    if ',' in path:
        path = path.split(',')
    
    if header:
        df = sparksession.read.options(header='True',inferSchema='True').csv(path)
    else:      
        df = sparksession.read.options(header='False',inferSchema='True').csv(path).toDF(*headerList)

    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def load_JSON(sparksession, path=[]): 
    
    df = sparksession.read.option("multiline", "true").json(path)
    
    return df    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def readConfigFromURL(url):    
    r = requests.get(url, allow_redirects=True)
    config = configparser.ConfigParser()
    config.read_string(r.text)
    return config

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def S3Exists(S3BucketAndKey):
    s3 = boto3.resource('s3')
    match = re.match(r's3:\/\/(.+?)\/(.+)', S3BucketAndKey)
    bucket = s3.Bucket(match.group(1))
    s3file = match.group(2)
    objs = list(bucket.objects.filter(Prefix=s3file))
    return any([w.key == s3file for w in objs])       

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def dataIngestionS3toS3(S3Dest, listOfOrign=[], preKey="", overWrite=False):
    s3 = boto3.resource('s3')
    return_list = []    
    
    for l in listOfOrign:
        match = re.match(r's3:\/\/(.+?)\/(.+)', l)
        copy_source = {
            'Bucket': match.group(1),
            'Key': match.group(2)
         }
        
        destFile = preKey + "/" + copy_source["Key"]        
       
        
        if S3Exists(S3Dest + "/" + destFile)==False or overWrite:
            s3.meta.client.copy(copy_source, S3Dest.replace("s3://",""), destFile)

        return_list.append(S3Dest + "/" + destFile)

    return return_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def dataIngestionHTTPtoS3(S3Dest, listOfOrign=[], preKey="",overWrite=False):
    s3 = boto3.client('s3')
    return_list = []    
    
    for l in listOfOrign:
        req = requests.get(l)
        req_data = [line.decode() + '\n' for line in req.iter_lines()]
        req_data = "".join(req_data)
        
        
        if 'content-disposition' in req.headers:
            d = req.headers['content-disposition']
            filename = re.findall("filename=(.+)", d)[0]
            filename = filename.replace('"','')
        else:
            filename = l.split("/")[-1]               

        destFile = preKey + "/" + filename
 
        
        if S3Exists(S3Dest + "/" + destFile)==False or overWrite:
            s3.put_object(Bucket=S3Dest.replace("s3://",""), Key=destFile, Body=req_data)
    
        return_list.append(S3Dest + "/" + destFile)
        
    return return_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def saveDFtoParquetS3(df, S3Dest, partitionBy=[], overWrite=False):
    if overWrite or S3Exists(S3Dest) == False:
        df.write.parquet(S3Dest, partitionBy=partitionBy, mode='Overwrite')    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
def dataQuality(df):
    #corrects any column name
    newDf= df.toDF(*(re.sub(r'[\.\s]+', '_', c) for c in df.columns))    
        
    #take off any empty rows
    newDf = newDf.dropna(how="all") 
    
    return newDf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def dataPrepGreenCabDF(df):
    #Data Dquality
    newDf = dataQuality(df)
  
    
    #Select only the columns we are going to use
    #cast datetime
    #2014-04-01 00:00:00
    newDf = newDf.select(F.to_timestamp(newDf["lpep_pickup_datetime"], "yyyy-MM-dd HH:mm:ss").alias("DT"))
    newDf = newDf.select("DT", F.year("DT").alias('Year'), F.month("DT").alias('Month'), F.dayofmonth("DT").alias('Day'), F.dayofweek("DT").alias("Day_Of_Week"), F.hour(newDf["DT"]).alias("Hour"), F.minute(newDf["DT"]).alias("Minute"), F.lit("Green Cab").alias("type"))
    
       
    return newDf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def dataPrepYellowCabDF(df):
    #Data Dquality
    newDf = dataQuality(df)
        
    #Select only the columns we are going to use
    #cast datetime    
    newDf = newDf.select(F.to_timestamp(newDf["_pickup_datetime"], "yyyy-MM-dd HH:mm:ss").alias("DT"))
    newDf = newDf.select("DT", F.year("DT").alias('Year'), F.month("DT").alias('Month'), F.dayofmonth("DT").alias('Day'), F.dayofweek("DT").alias("Day_Of_Week"), F.hour(newDf["DT"]).alias("Hour"), F.minute(newDf["DT"]).alias("Minute"), F.lit("Yellow Cab").alias("type"))
    
    
    return newDf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def dataPrepUber(df):
    #Data Dquality
    newDf = dataQuality(df)
    
      
    #Select only the columns we are going to use
    #cast datetime
    if "Pickup_date" in newDf.columns:
        date_field = "Pickup_date"  
        date_format = "yyyy-MM-dd HH:mm:ss"
    else:
        date_field = "Date/Time"
        date_format = "MM/dd/yyyy HH:mm:ss"
    
    
    newDf = newDf.select(F.to_timestamp(newDf[date_field], date_format).alias("DT"))        
    newDf = newDf.select("DT", F.year("DT").alias('Year'), F.month("DT").alias('Month'), F.dayofmonth("DT").alias('Day'), F.dayofweek("DT").alias("Day_Of_Week"), F.hour(newDf["DT"]).alias("Hour"), F.minute(newDf["DT"]).alias("Minute"), F.lit("Uber").alias("type"))
    
    
    return newDf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def dataPrepWeather(df):
    #Data Dquality
    newDf = dataQuality(df)    

    newDf = newDf.select(F.to_timestamp(F.from_unixtime(F.unix_timestamp(F.concat(F.to_date(newDf["Date"]), F.lit(" "), newDf["Time"]),'yyyy-MM-dd hh:mm aa'),"MM/dd/yyyy HH:mm"),"MM/dd/yyyy HH:mm").alias("DT"), "Condition", "Station_Name")    
    newDf = newDf.select("DT", F.year("DT").alias('Year'), F.month("DT").alias('Month'), F.dayofmonth("DT").alias('Day'), F.dayofweek("DT").alias("Day_Of_Week"), F.hour(newDf["DT"]).alias("Hour"), "Condition", "Station_Name")    
    
    #DATA QUALITY - There are more than 1 weather condition on the sme hour. Take the first one
    window = Window.partitionBy("Year", "Month", "Day", "Hour", "Station_Name").orderBy("DT")
    newDf = newDf.withColumn("row_number",F.row_number().over(window))
    newDf=  newDf.filter(newDf["row_number"]==1).drop("DT", "row_number")
     
    return newDf
   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
def dataPrepCrash(dF):
    #Data Dquality
    newDf = dataQuality(dF)
    
    newDf = newDf.withColumn("DT",F.to_timestamp(F.concat(F.to_date(F.to_timestamp(F.col("CRASH_DATE"))),F.lit(" "), F.col("CRASH_TIME")), "yyyy-MM-dd HH:mm") )
    
    newDf = newDf.select("DT",
                         F.year("CRASH_DATE").alias("Year"), \
                         F.month("CRASH_DATE").alias("Month"), \
                         F.dayofmonth("CRASH_DATE").alias("Day"), \
                         F.dayofweek("CRASH_DATE").alias("Day_Of_Week"), \
                         F.hour("CRASH_TIME").alias("Hour"), \
                         F.minute("CRASH_TIME").alias("Minute"))
    newDf = newDf.filter(newDf["Year"] != 2020)
    
    return newDf
    
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
def mergeDataFrames(iniDf, DataFrame=[]):
    for d in DataFrame:
        iniDf.union(d)
    return iniDf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
def dataTransformation(spark, config, dataLoadedDictDF):
    """   
    - DATA TRANSFORMATION
    
    - Loads data from the Blob Area
    
    - Do some Quality Checks and Save the transformed files into de DATA STORAGE Area, in parquet format
    
    - Creates the final Analysis Dataframes and stores in the DATA STORAGE
    """
    #Data Cleansing / Data Prep
    yellowCab = dataPrepYellowCabDF(dataLoadedDictDF["yellowCab"]) 
    greenCab  = dataPrepGreenCabDF(dataLoadedDictDF["greenCab"])    
    uber14    = dataPrepUber(dataLoadedDictDF["uber14"])    
    uber15    = dataPrepUber(dataLoadedDictDF["uber15"])    
    weather   = dataPrepWeather(dataLoadedDictDF["weather"])    
    crash     = dataPrepCrash(dataLoadedDictDF["crash"])
          
    
    storage = config.get('DATASTORAGE','DATASTORAGE')
    storageFolder = config.get('DATASTORAGE','DATASTORAGE_FOLDER')

    #Saves Data to Data Storage
    saveDFtoParquetS3(yellowCab, storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','YELLOWCABPARQUET'), ["Year", "Month"])
    saveDFtoParquetS3(greenCab, storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','GREENCABPARQUET'), ["Year", "Month"])
    saveDFtoParquetS3(uber14, storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','UBER14PARQUET'), ["Year", "Month"])
    saveDFtoParquetS3(uber15, storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','UBER15PARQUET'), ["Year", "Month"])
    saveDFtoParquetS3(weather, storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','WEATHERPARQUET'), ["Year", "Month"])
    saveDFtoParquetS3(crash, storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','CRASHPARQUET'), ["Year", "Month"])
 
    #CREATES FINAL DATAFRAMES FOR ANALYSIS
    ## PICKUPS DATAFRAME FOR ANALYSIS
    allpickups = mergeDataFrames(yellowCab, [greenCab,uber14, uber15])        
   
    ## SAVES MERGED PICKUPS DATAFRAME TO DATASTORAGE
    saveDFtoParquetS3(allpickups, storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','ALLPICKUPSPARQUET'), ["Year", "Month"])
       
    
    ##JOIN DATA QUALIT. COLLECT ORIGINAL ROWS COUNT    
    dtQualityBeforeJoinRowsCount = allpickups.count()

    start5 = timer()
    station01 = weather.filter(weather["Station_Name"] == "John F. Kennedy International Airport Station").dropDuplicates()
    station02 = weather.filter(weather["Station_Name"] == "LaGuardia Airport Station").dropDuplicates()

    
    analysisPickupsDf = allpickups.join(station01, on=["Year","Month","Day","Hour"],how='left')\
                                         .join(station02, on=["Year","Month","Day","Hour"],how='left')\
                                         .select(allpickups['*'],station01['Condition'].alias("JFK_Condition"),\
                                             station02['Condition'].alias("LGA_Condition"))
 
    
    ##COLLECT ROWS COUNT AFTER JOIN
    ##ROWS BEFORE AND AFTER JOIN MUST BE THE SAME
    ##SAVES DATA DO DATA STORAGE    
    dtQualityAfterJoinRowsCount = analysisPickupsDf.count()
    if dtQualityBeforeJoinRowsCount == dtQualityAfterJoinRowsCount:
        saveDFtoParquetS3(analysisPickupsDf, storage + "//" + storageFolder + "/" + config.get('DATASTORAGE','ANALYSISPICKUPSPARQUET'), ["Year", "Month"])
    else:
        print("Error joing tables. Before joing dataframe {} had {} rows. After {}".format("analysisPickupsDf", dtQualityBeforeJoinRowsCount, dtQualityAfterJoinRowsCount))

    ## CRASH DATAFRAME FOR ANALYSIS
    analysisCrashsDf = crash        

    ##JOIN DATA QUALIT. COLLECT ORIGINAL ROWS COUNT
    dtQualityBeforeJoinRowsCount = analysisCrashsDf.count()
    start6 = timer()
    analysisCrashsDf = analysisCrashsDf.join(station01, on=["Year","Month","Day","Hour"],how='left')\
                                       .join(station02, on=["Year","Month","Day","Hour"],how='left')\
                                       .select(analysisCrashsDf['*'],station01['Condition'].alias("JFK_Condition"),\
                                             station02['Condition'].alias("LGA_Condition"))
  
    
    ##COLLECT ROWS COUNT AFTER JOIN
    ##ROWS BEFORE AND AFTER JOIN MUST BE THE SAME
    ##SAVES DATA DO DATA STORAGE
    dtQualityAfterJoinRowsCount = analysisCrashsDf.count()
    if dtQualityBeforeJoinRowsCount == dtQualityAfterJoinRowsCount:
        saveDFtoParquetS3(analysisCrashsDf, storage + "//" + storageFolder + "/" + config.get('DATASTORAGE','ANALYSISCRASHSPARQUET'), ["Year", "Month"])
    else:
        print("Error joing tables. Before joing dataframe {} had {} rows. After {}".format("analysisCrashsDf", dtQualityBeforeJoinRowsCount, dtQualityAfterJoinRowsCount))

       
    return {"yellowCab" : yellowCab,
            "greenCab": greenCab,
            "uber14": uber14,
            "uber15": uber15,
            "weather": weather,
            "crash": crash,
            "analysisPickupsDf": analysisPickupsDf,
            "analysisCrashsDf": analysisCrashsDf,
            "allpickups": allpickups
       }


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def dataExtration(spark, config, newExtraction=True):
    """   
    - DATA Extraction
    
    - Ingest all the data to STORAGE BLOB
    
    - Ingestions from S3 (csv files) and a JSON (API Endpoint)
    
    - If newExtraction is set to True, do ingestion despite the fact that we alrealy have the file in the STORAGE BLOB
      otherwise return the STORAGE BLOB file path
    
    """    
    storageBlob =  config.get('STORAGE_BLOB','STORAGE_BLOB')
    storageBlobFolder = config.get('STORAGE_BLOB','STORAGE_BLOB_FOLDER')

    yellowCabIngestedData = dataIngestionS3toS3(storageBlob, config.get('DATA_SOURCES','YELLOW_CAB').split(","), storageBlobFolder, newExtraction)
    greenCabIngestedData  = dataIngestionS3toS3(storageBlob, config.get('DATA_SOURCES','GREEN_CAB').split(","), storageBlobFolder, newExtraction)
    uber14IngestedData    = dataIngestionHTTPtoS3(storageBlob, config.get('DATA_SOURCES','UBER14').split(","), storageBlobFolder, newExtraction)
    uber15IngestedData    = dataIngestionHTTPtoS3(storageBlob, config.get('DATA_SOURCES','UBER15').split(","), storageBlobFolder, newExtraction)
    weatherIngestedData   = dataIngestionHTTPtoS3(storageBlob, config.get('DATA_SOURCES','WEATHER_WUNDERGROUND').split(","), storageBlobFolder,  newExtraction)
    crashIngestedData     = dataIngestionHTTPtoS3(storageBlob, config.get('DATA_SOURCES','CRASH').split(","), storageBlobFolder, newExtraction)
    
    
    return {"yellowCabIngestedData" : yellowCabIngestedData,
            "greenCabIngestedData": greenCabIngestedData,
            "uber14IngestedData": uber14IngestedData,
            "uber15IngestedData": uber15IngestedData,
            "weatherIngestedData": weatherIngestedData,
            "crashIngestedData": crashIngestedData
           }
    end = timer()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def dataLoadFromParquet(parquetFile):
    schema = StructType([StructField('none', StringType(), True)])
    return spark.read.parquet(parquetFile) if S3Exists(parquetFile+"/_SUCCESS") else spark.createDataFrame([], schema)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
def dataLoadFromDataStorage(spark, config):
    """   
    - Loads Transformed Data From DATA STORAGE   
  
    """
    storageBlob =  config.get('STORAGE_BLOB','STORAGE_BLOB')
    storageBlobFolder = config.get('STORAGE_BLOB','STORAGE_BLOB_FOLDER')        
    storage = config.get('DATASTORAGE','DATASTORAGE')
    storageFolder = config.get('DATASTORAGE','DATASTORAGE_FOLDER')
    
    #Get the already transformed parquet data from the Storage
    yellowCab =         dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','YELLOWCABPARQUET')) 
    greenCab =          dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','GREENCABPARQUET')) 
    uber14 =            dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','UBER14PARQUET')) 
    uber15 =            dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','UBER15PARQUET')) 
    weather =           dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','WEATHERPARQUET')) 
    crash =             dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','CRASHPARQUET')) 
    analysisPickupsDf = dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','ANALYSISPICKUPSPARQUET')) 
    analysisCrashsDf =  dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','ANALYSISCRASHSPARQUET')) 
    allpickups =        dataLoadFromParquet(storage + "/" + storageFolder + "/" + config.get('DATASTORAGE','ALLPICKUPSPARQUET')) 
    
    
    return {"yellowCab" : yellowCab,
        "greenCab": greenCab,
        "uber14": uber14,
        "uber15": uber15,
        "weather": weather,
        "crash": crash,
        "analysisPickupsDf": analysisPickupsDf,
        "analysisCrashsDf": analysisCrashsDf,
        "allpickups": allpickups
   }

 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
def dataLoad(spark, config, dataExtractedDict):
    """   
    - Loads Data From STORAGE BLOB into memory   
  
    """
    #Load Data in memory from STORE BLOB
    yellowCab = load_csv(spark, dataExtractedDict["yellowCabIngestedData"])
    greenCab = load_csv(spark, dataExtractedDict["greenCabIngestedData"])
    uber14 = load_csv(spark, dataExtractedDict["uber14IngestedData"])     
    uber15 = load_csv(spark, dataExtractedDict["uber15IngestedData"])     
    weather = load_csv(spark, dataExtractedDict["weatherIngestedData"])
    crash = load_JSON(spark, dataExtractedDict["crashIngestedData"])
   
    return {"yellowCab" : yellowCab,
        "greenCab": greenCab,
        "uber14": uber14,
        "uber15": uber15,
        "weather": weather,
        "crash": crash
       }


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
def main():
    """   
    - Gets the config info from confog.cfg
    
    - Creates Spark Session
    
    - Loads Data from S3 Bucket and saves it to another S3 Bucket
    
    """
    start_total = timer()
    
    #loads config    
    config = configparser.ConfigParser()
    config = read("config.cfg")
       
    #Creates Spark Session
    spark = create_spark_session()
  
    #EXTRACTION (Data Sourcer --> Storage BLOB)
    start = timer()
    dataExtractedDict = dataExtration(spark, config, config.get('GENERAL','RUN_EXTRATION') == "True")
    end = timer()
    print("dataExtration: ",timedelta(seconds=end-start))
       
    
    #LOAD (Storage BLOB --> Memory)
    start = timer()
    dataLoadedDictDF = dataLoad(spark, config, dataExtractedDict) if config.get('GENERAL','RUN_LOAD') == "True" else {}
    end = timer()
    print("dataLoad: ",timedelta(seconds=end-start))
    
    #TRANSFORMATION
    #If transformation is set to True and Data is Loaded into memory, run transformation
    #Load final dataframes otherwise
    start = timer()
    transformation_condition = config.get('GENERAL','RUN_TRANSFORMATION') == "True" and config.get('GENERAL','RUN_LOAD') == "True"
    dataTransformedDictDf = dataTransformation(spark, config, dataLoadedDictDF) if transformation_condition else dataLoadFromDataStorage(spark, config)
    end = timer()
    print("dataTransformation: ",timedelta(seconds=end-start))
    
    #Show dataframes created for analysis
    dataTransformedDictDf["analysisPickupsDf"].printSchema()       
    dataTransformedDictDf["analysisCrashsDf"].printSchema()
    
    end_total = timer()
    print("MAIN - Execution Total Time: ",timedelta(seconds=end_total-start_total))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
if __name__ in ("__main__","builtins"):
    main()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataExtration:  0:19:12.580805
dataLoad:  0:03:26.804214
dataTransformation:  0:18:29.819729
root
 |-- DT: timestamp (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Day_Of_Week: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- type: string (nullable = false)
 |-- JFK_Condition: string (nullable = true)
 |-- LGA_Condition: string (nullable = true)

root
 |-- DT: timestamp (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Day_Of_Week: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- JFK_Condition: string (nullable = true)
 |-- LGA_Condition: string (nullable = true)

MAIN - Execution Total Time:  0:41:10.022905